In [5]:
import pytorch_lightning as pl
import h5py
import os
import torch
import csv
import re
import numpy as np
import sqlite3
from torch import nn
from torch.nn import functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import library as lib
import pandas as pd
from matplotlib.pylab import plt
from sklearn.manifold import trustworthiness

In [6]:
pl.seed_everything(42, workers=True) 

Global seed set to 42


42

# Parameters

In [7]:
work_dir = "./"
inputs = "./chumby_data/phosphatase.db"
out_path = "./chumby_data/phoshatase_out/"

writer = True
gpu = True

epoch = 1

dim = 2560
dim3 = int(dim * 1/32)
dim2 = int(dim * 1/16)
dim1 = int(dim * 1/4)

In [55]:
#read in DB file into headers and embedding format



NameError: name 'DATABASE' is not defined

In [ ]:
#make numpy file with (seq header, embedding)
esm2 = h5py.File(inputs+"kinase_esm2_3b.h5", 'r')
list(esm2.keys())
# List all groups
keys = list(esm2.keys())
#print(keys) 
# Get the data
embedding = []
headers = []
for key in keys:
    #data
    emb = esm2[key][:]
    embedding.append(emb)
    #headers
    key = key.replace("isoform=", "isoform_").replace("=", "").replace(":", "_")
    if key == "GZMA_Canis_lupus":
        print("header is now changed")
        key = "GZMA_Canis_lupus_1"
    headers.append(key)

headers = np.array(headers)
embedding = np.array(embedding)
#replace whitespaces
new_headers = []
for line in headers:
    line = line.replace(' ', '_')
    new_headers.append(line)
headers = np.array(new_headers)

# Data Pre-Processing

In [26]:
#split train test
headers_first, headers_test, embedding_first, embedding_test = train_test_split(headers, embedding, test_size=0.1, random_state=42)
#split train validation
headers_train, headers_validate, embedding_train, embedding_validate = train_test_split(headers_first, embedding_first, test_size =0.2, random_state=42)
print("total:", headers.shape)
print("training+validation:", headers_first.shape)
print("test:", headers_test.shape)
print("training:", headers_train.shape)
print("validation:", headers_validate.shape)
print(len(headers_train), len(embedding_train))
print(len(headers_validate), len(embedding_validate))

#training_df = pd.DataFrame({'header': headers_train, 'embedding' : embedding_train})
#validate_df = pd.DataFrame({'header': headers_validate, 'embedding' : embedding_validate})

total: (204,)
training+validation: (183,)
test: (21,)
training: (146,)
validation: (37,)
146 146
37 37


# Dataloaders

In [27]:
train_dataset = lib.makedataset(headers_train, embedding_train)
val_dataset = lib.makedataset(headers_validate, embedding_validate)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0, drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=0, drop_last=True)
    

# Training VAE

In [29]:
#if prott5 used: dtype=float16 -> error with .linear()

#encoder_layers, latent_dim, decoder_layers


encoder_layers = [dim, dim1, dim2, dim3]
latent_dim = dim3
decoder_layers = [dim3, dim2, dim1, dim]

# Initialize the VAE model
vae = lib.VariationalAutoencoder(encoder_layers, latent_dim, decoder_layers)

if gpu:
    trainer = pl.Trainer(accelerator="gpu", devices = 1, max_epochs=epoch, log_every_n_steps=4, check_val_every_n_epoch=1)
else:
    trainer = pl.Trainer(accelerator="auto", max_epochs=epoch, log_every_n_steps=4, check_val_every_n_epoch=1)

# Train the VAE model
trainer.fit(vae, train_dataloader, val_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 1.8 M 
1 | latent  | Sequential | 6.6 K 
2 | fc21    | Linear     | 6.5 K 
3 | fc22    | Linear     | 6.5 K 
4 | fc3     | Sequential | 6.6 K 
5 | fc4     | Sequential | 6.6 K 
6 | relu    | ReLU       | 0     
7 | decoder | Sequential | 1.8 M 
---------------------------------------
3.6 M     Trainable params
0         Non-trainable params
3.6 M     Total params
14.211    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

C:\Users\alex\anaconda3\envs\FP_Lightning\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\alex\anaconda3\envs\FP_Lightning\lib\site-packages\pytorch_lightning\utilities\data.py:103: UserWarning: Total length of `DataLoader` across ranks is zero. Please make sure this was your intention.
  rank_zero_warn(
C:\Users\alex\anaconda3\envs\FP_Lightning\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init

Training: 0it [00:00, ?it/s]

TypeError: can't convert np.ndarray of type numpy.bytes_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

# Fix Formatting

In [7]:
def find_largest_number(root_dir):
    largest_number = 0
    for subdir, dirs, files in os.walk(root_dir):
        for dir_name in dirs:
            match = re.search(r'\d+', dir_name)
            if match:
                current_number = int(match.group())
                if current_number > largest_number:
                    largest_number = current_number
    return largest_number

largest_num = find_largest_number("./lightning_logs")

#Fix formatting
with open('lightning_logs/version_' + str(largest_num) + '/metrics.csv', 'r') as file:
    reader = csv.reader(file)
    rows = list(reader)

combined_rows = []
header_row = ['train_loss','reconstruction_loss','kl_loss','epoch','step','validation_loss']

combined_rows.append(header_row)
for i in range(1, len(rows) -1, 2):
    train_loss = rows[i][0]
    reconstruction_loss = rows[i][1]
    kl_loss = rows[i][2]
    epoch = rows[i+1][3]
    step = rows[i+1][4]
    validation_loss = rows[i+1][5]
    combined_rows.append([train_loss,reconstruction_loss,kl_loss,epoch,step,validation_loss])

with open('lightning_logs/version_' + str(largest_num) + '/new_metrics.csv', 'w', newline = "") as file:
    writer = csv.writer(file)
    writer.writerows(combined_rows)

FileNotFoundError: [Errno 2] No such file or directory: 'lightning_logs/version_95/metrics.csv'

# Plot Loss

In [ ]:
#Plot
# read csv file
df = pd.read_csv('lightning_logs/version_' + str(largest_num) + '/new_metrics.csv')

df1 = df[df['epoch'] >= int(epoch)/2]

# plot 0
plt.plot('epoch', 'train_loss', data=df)
plt.plot('epoch', 'reconstruction_loss', data=df)
plt.plot('epoch', 'validation_loss', data=df)
plt.title('Epoch vs Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['loss', 'reconstruction loss', 'validation_loss'], loc='upper right')
plt.show()


# plot 1
plt.plot('epoch', 'train_loss', data=df1)
plt.plot('epoch', 'reconstruction_loss', data=df1)
plt.plot('epoch', 'validation_loss', data=df1)
plt.title('Epoch vs Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['loss', 'reconstruction loss', 'validation_loss'], loc='upper right')
plt.show()

# Encode

In [8]:
encode = vae.encoder(torch.Tensor(embedding))
#mu, logvar = vae.encode(torch.Tensor(embedding))
#en = vae.reparameterize(mu, logvar)
#encode = vae.decode(en)
print(encode)

#encode = encode #[1]


tensor([[0.3105, 0.6995, 0.2642,  ..., 0.1019, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.9912, 0.0000],
        [0.0000, 0.5824, 0.0000,  ..., 0.0000, 0.0000, 0.0247],
        ...,
        [0.0000, 1.5785, 1.4250,  ..., 0.0000, 0.0221, 0.5507],
        [2.1244, 1.2211, 1.8740,  ..., 0.0000, 0.0865, 0.4631],
        [0.0000, 1.4073, 0.0000,  ..., 0.0000, 0.4206, 0.0000]],
       grad_fn=<ReluBackward0>)


# Neighbor_joining and create tree

In [9]:
# Neighbor_joining and create tree

#cosine, euclidean, manhattan, ts_ss, jensenshannon
metric_list = ["cosine","euclidean", "manhattan", "ts_ss"]

for metric in metric_list:
    out_newick = out_path+"kinase_esm2_nj_reference"+str(epoch)+"_"+metric+".newick"

    embedding_out = encode.detach() #torch.stack(encode).detach() 
    #print(embedding_out)

    #distance metric
    dist = lib.distance_metric(embedding_out)
    distmat = dist.get_metric(embedding_out, metric) 
    
    #neighbor joining to tree
    nj = lib.neighbor_joining(distmat,headers)
    newick = nj.get_newick(distmat,headers)
    
    #trustworthiness
    _distmat  = lib.cophenetic_distmat(newick, names=headers)
    _trustworthiness = trustworthiness(distmat, _distmat, n_neighbors=10, metric='precomputed')
    print("trustworthiness: ", _trustworthiness)
    
    
    #silhouette = silhouette_score(_distmat, headers, metric='precomputed')
    #silhouette = lib.silhouette.get_silhouette(distmat, headers)
    #print("silhouette: ", silhouette)
    

    if writer:
        with open(out_newick, 'w') as w:
            w.write(newick)
            w.close()

trustworthiness:  0.9387387476669474
trustworthiness:  0.8096714731678312
trustworthiness:  0.8452828568125135
trustworthiness:  0.8353917050691244


# UPGMA and create tree

In [10]:
#write output flag
from sklearn.metrics import silhouette_score
from itertools import groupby
import string


#cosine, euclidean, manhattan, ts_ss, jensenshannon
metric_list = ["cosine","euclidean", "manhattan", "ts_ss"]

for metric in metric_list:
    out_newick = out_path+"kinase_esm2_upgma_reference"+str(epoch)+"_"+metric+".newick"

    embedding_out = encode.detach() #torch.stack(encode).detach() 
    #print(embedding_out)
    
    #distance metric
    dist = lib.distance_metric(embedding_out)
    distmat = dist.get_metric(embedding_out, metric)
    #print(distmat)
    
    #upgma to tree
    upgma = lib.upgma(distmat,headers)
   
    #trustworthiness
    _distmat  = lib.cophenetic_distmat(upgma, names=headers)
    _trustworthiness = trustworthiness(distmat, _distmat, n_neighbors=10, metric='precomputed')
    print("trustworthiness: ", _trustworthiness)
    
    
    if writer:
        with open(out_newick, 'w') as w:
            w.write(upgma)
            w.close()

trustworthiness:  0.9412034421815899
trustworthiness:  0.8543144541895843
trustworthiness:  0.8845779033083924
trustworthiness:  0.8792461556249278
